# Regular expressions

For the past two weeks we've been learning how to extract text out of XML documents via XPath statements.  Now we're going to turn our attention to pure text.  So whereas XPath statements describe patterns of locations in XML documents, regular expressions describe how text might look.

For example, if we're looking for instances in a document where a year is noted (e.g. 2017), we could search for the year we expected.  But what if there a range of years?  We could search for "201" to catch 2010-2019, but we'd need to switch patterns for anything outside that range.  Likewise, if you're looking for a year anywhere between 1900-1999 you could search for just "19" but now you might get ages, days, or other numerical values.

This kind of search is exactly what regular expressions are designed to do.  Instead of starting with constituant numbers, we could build up a more specific pattern of what that year might look like.  

For the range: 2010-2019:

* We know that a year is composed of four integer values all together. Yes, sometimes years might be in "'nn" or "nn" format, but let's say there will be 4 numbers in the document we're searching on.
* We know that the first three numbers will be constant:  a literal 2, a literal 0, and a literal 1.  
* The last number is what can vary, and we can say it is any integer value between 0 and 9.

These three rules can be described within a regular expression. 

This is a good time to skip off and do some reading before continuing.  You should be reading chapter 11 of PfE this week, but there's another source you can look for.  Our friendly relevant Wikipedia page.  https://en.wikipedia.org/wiki/Regular_expression  You can skim some of the narrative for background, but I'd like you to do more focused reading on:

* Basic concepts: https://en.wikipedia.org/wiki/Regular_expression#Basic_concepts
* Formal langugae: https://en.wikipedia.org/wiki/Regular_expression#Formal_language_theory
    * Gloss over the math notation, focus on the examples and context
* Syntax:  https://en.wikipedia.org/wiki/Regular_expression#Syntax
    * Head for that table and focus on reading the examples rather than understanding the technical jargon.
    
    
Many software programs dealing with text have support for regular expression searches.  Even PyCharm!  When you open up a text file, start a search inside of it, and you'll see a check box for Regex.  So you can practice on this without needing to use Python.

In fact, that's what I recommend.  Particularly when you're trying to do known item searchers within a text document, you often want to iteratively experiment with your expressions directly on the document before you bring it into python to extract those results.  This will give you instant feedback of what it is finding.

For example, say you have a document with 100 records.  The data is semi-structured, so you've decided to use a regular expression to extract out a certain data point.  As your query, there will be a result count.  When you think you have your expression done, check the count.  If you see something other than 100, you know that you need to change it.  

* A number less than your known result count means that you've made your expression too restrictive.  You're falsely rejecting some data.  
* A number more than your known result count means that you've made your expression too permissive.  You're flasely accepting some data.

There will be times that you cannot get exactly what you need with a single regular expression.  That's usually because the data is too unstructured and the rules are too complex or broad to be applied over the entire document.  This is usually a good point, and a valuable place, to open your string processing skills to subdivide the document.  For example, if you have a very broad search, you might want to do make subdivisions (remember splitting Dracula apart?) and apply the broad search to just the sections that you know apply.

For example, say you have a long report of 100,000 summary records in one document.  These are records on snake species and their field measurements.  You want the length field, which is present within each record, but you only want it for the boa species.  This is a pretty classic data query, and maybe you can imagine how easy it would be to construct in SQL.  But instead of a lovely database, you have an unlovely semistructured text report.

So trying to write a regular expression to get the length value is going to be overly permissive.  You're going to get it for all 100,000 records.  But say you know that there are 45,000 boa records.  You know this because you've also run a regular expression to detect how many species records are classified as boa.  It might be possible to include that subdivision in your regular expression, but there's a good chance it'll be so complex and unweidly that you won't be able to contain it or make a good slice.  But, you very likely could split the document apart, such that you have all 100,000 records as separate strings, then you can filter out just the boa records, and then apply your length expression to just that subset of strings.  

This kind of situation is when using regular expressions in the context of Python is very valuable.  

There are also situations when using other mixed methods, such as a combination of regular expressions and xpath statements make a lot of sense.  For example, when doing web scraping, sometimes there will be fields contained in single HTML elements that actually have multiple data points.  Those data points are only separated by text delimiters (words or symbols).  So you use xpath to cleanly extract the element text value, and then you throw that text into regular expressions for splitting.  This is another example of subsetting known data before feeding it into regular expressions.  The more you can clean away noise from your source text before applying a regular expression, the better.

# Regular expressions in PyCharm

Let's practice a few expressions on our ab-104.xml document.  Yes, this is an xml document, but it is just plain text so we can do whatever we want with it.

## finding a year

Let's practice our year query first.  We're going to use a character class here to specify the numbers that are applicable.  Character classes are contained within `[]`.  You can specify `[a-z]`, `[A-Z]`, `[abcxyz]`, etc. Any combination of literal characters can go there, and you can do ranges based on the ascii table.

### Basic numerical class

So in the search, with Regex checked, place `[0-9]`.  You get 225 matches, because we're matching every single number within the document.  

### Repetition

But recall that we said that years have 4 numbers.  We can use the `{}` directly after it to specify a repetition.  This takes two arguments, first the minimum number of times and the second the maximum number of times it could match.  Or you can use it with a single argument, specifying the exact number of matches.  That's what we want to do in this case.

There are many ways to declare repetitions, and they all operate on the element immediately preceding this metacharacter:

* `?` will match that element 0 or 1 times
    * Example:  "cats?" will match cat or cats
* `*` will match that element 0-infinity times
    * Example: "cats*" will match cat, cats, or catssssssssssssssssss (etc.)
* `+` will match that element 1-infinity times
    * Example:  cats+ will match cats or catssssssssssss (etc.)
* `{}` can be used in two ways:
    * `{min_times, max_times}` will match that element at least min_times but no more than max_times
    * `{exact_number_of_times}` will match that element the specific number of times declared 
    * Example: cats!{1,3} will match cats!, cats!!, or cats!!! and cats!{2} will match only cats!!

`[0-9]{4}` means "match any integer between 0 and 9 (inclusive) in a group 4 times."  So this will find any group of 4 consecutive integer numbers.  This gives us 38 matches.  But it seems improbable that we'd have 38 years within this document.  So let's scroll through them.  We're getting numbers from address lines for campus mailing unit numbers, partial matches on 5 digit zip codes, matches on the 4 digit sub zip code, a partial match for the year in a yyyymmdd date in <when>, and then the mmdd section in that same date, the year match in a yyyy-mm-dd date in the <revisionDesc> <when> element, and so on. 

## That was a lot of matches

This helps us explore a bit of the implections behind our data design and the limitations of regular expressions.  This is a good example of how much of a brute force tool this is.  So how can refine things a bit more?  Depending on our data question, we may not be able to.  

But we can do a little bit.  We can build up potentially more restrictive expressions.  Right now we're matching years from any millenia and century after the year 1000.  Maybe we can say that we only want years between 1900 and 2099.  This seems like a good first place to refine things.

## Adding more speficicity

Comparing the two years on our end range, we can say this "A year is something that starts with a 19 or 20, followed by exactly 2 integers." We can't really say "after 80" because we're dealing with 00-17 for current years.  We can explicitly state certain values by placing them directly in the regex expression.  This is much the same as searching literally for the year "1999" with an expression of "1999".  

But if we wanted to search for anything between 1990 and 1999, we can combine both our literals and our repetition of integer values.  So '19' will match the literal value of 1 and literal value of 9.  Then we can do '[0-9]{2}' to match two consecutive integers between 0 and 9. Combining these two, we could say '19[0-9]{2}'.

This gets us a little closer to the problem, because we have '19[0-9]{2}' or '20[0-9]{2}'.  How can we combine these?

### Saying "or"

The pipe `|` symbol is how we specify or.  BEWARE, "or" is very very aggressive.  If you make a pattern that says something like 'stuff|yoiamotheroptionsandstuffallovertheplace', it'll go entire for "stuff" or "yoiamotheroptionsandstuffallovertheplace".

So if we try to use an expression like '19|20[0-9]{2}', it will match literal only '19' or '20[0-9]{2}'  We can fix this by containing our options with ().  The pip symbol will reach out to an open or closing paren, so we need to enclose our "or" items within parens.  In this case, just the '19|20' values are alternating.

So we can now look at the results of (19|20)[0-9]{2}.  

# Is that it?

There's a lot to regular expressions, but character classes, (), |, and a variety of repetitions are pretty much 

There are many other symbols, including those to declare that a match must start at the beginning of the line, be against the end of a line, etc.  

# Regex in python

There's the lovely `re` module with all kinds of nifty features.  But as you'll learn in your homework, the regex is only half the battle.  The basic pattern is:

* have a string of text to do the searching within
* compile the pattern into a variable
* use `re.findall()` to get the matches
* do something interesting with the results

You can see more in the documentation (look for neat examples in the green boxes!): https://docs.python.org/3.6/library/re.html

Now, there are functions to detect if there's any matches in something, doing substitutions, etc.

## `re.findall()` 

This function will usually take two arguments: 1) the regex pattern, 2) the text source (as a string).

In [3]:
import re

with open('xmldocs/ab-104.xml', 'r') as file_in:
    text = file_in.read() # now I have all the text in as a string
    
year_pattern = '[0-9]{4}'

year_results = re.findall(year_pattern, text)

print(year_results)

['8211', '6858', '4100', '6858', '4100', '2013', '0718', '2013', '2012', '2012', '2012', '2012', '1950', '2009', '2216', '2012', '2012', '2013', '2012', '2011', '2012', '2012', '2012', '2011', '2011', '2011', '2011', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2013', '2012']


We can see the results a bit easier now.  Including those extra 8211, etc.  Obviously not years.  So let's rebuild our refinements.

In [4]:
year_pattern = '19[0-9]{2}' # 1900-1999

year_results = re.findall(year_pattern, text)

print(year_results)

['1950']


In [6]:
year_pattern = '20[0-9]{2}' #2000-2999

year_results = re.findall(year_pattern, text)

print(year_results)

['2013', '2013', '2012', '2012', '2012', '2012', '2009', '2012', '2012', '2013', '2012', '2011', '2012', '2012', '2012', '2011', '2011', '2011', '2011', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2013', '2012']


We can actually further refine 2000-2019 if we want.

In [8]:
year_pattern = '20[0-1][0-9]'

# so:  a literal 2, a literal 0, then 0-1 once, and 0-9 once

year_results = re.findall(year_pattern, text)

print(year_results)

['2013', '2013', '2012', '2012', '2012', '2012', '2009', '2012', '2012', '2013', '2012', '2011', '2012', '2012', '2012', '2011', '2011', '2011', '2011', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2013', '2012']


In [14]:
year_pattern = '(19|20)[0-9]{2}' # and our final one

# so:  a literal 2, a literal 0, then 0-1 once, and 0-9 once

year_results = re.findall(year_pattern, text)

print(year_results)

['20', '20', '20', '20', '20', '20', '19', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20', '20']


But it doesn't work!!!  Kind of odd, yes, but this highlights some of the reflavoring in Python.  So there's a concept of a capturing group, also used with ().  But this is intermingled with how () are used to contain patterns.  Thus, we need to say "Hey, this isn't a matching group, ignore it and just treat it like a containment group."  We do that with `(?:pattern)`.

In [15]:
year_pattern = '(?:19|20)[0-9]{2}' # and our final one

# so:  a literal 2, a literal 0, then 0-1 once, and 0-9 once

year_results = re.findall(year_pattern, text)

print(year_results)

['2013', '2013', '2012', '2012', '2012', '2012', '1950', '2009', '2012', '2012', '2013', '2012', '2011', '2012', '2012', '2012', '2011', '2011', '2011', '2011', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2012', '2013', '2012']
